In [6]:
import torch as torch
import os
from torch import nn
    
class mlp_no_bias(nn.Module):
    def __init__(self, input_size, output_size=None):
        super().__init__()
        if(output_size is None):
            output_size = input_size
        self.linear = nn.Linear(input_size, input_size, bias=False)
        self.linear2 = nn.Linear(input_size, output_size, bias=False)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x
    
class linear_no_bias(nn.Module):
    def __init__(self, input_size, output_size=None):
        super().__init__()
        if(output_size is None):
            output_size = input_size
        self.linear = nn.Linear(input_size, output_size, bias=False)
    def forward(self, x):
        x = self.linear(x)
        return x

num_layers = 5
dir = "sparse_weights"
linear_name = "linear_weights_no_bias_{layer}.pt"
mlp_name = "mlp_weights_no_bias_{layer}.pt"
linear_sparse_connections = []
mlp_sparse_connections = []
for layer in range(num_layers):
    linear_sparse_connections.append(torch.load(os.path.join(dir, linear_name.format(layer=layer))))
    mlp_sparse_connections.append(torch.load(os.path.join(dir, mlp_name.format(layer=layer))))

In [9]:
threshold = 0.0005
for i in range (10):
    large_weights = (linear_sparse_connections[0].linear.weight[i] > threshold).sum()
    print(f"Large weights: {large_weights}")

Large weights: 5
Large weights: 6
Large weights: 3
Large weights: 6
Large weights: 13
Large weights: 21
Large weights: 3
Large weights: 3
Large weights: 4
Large weights: 3
